In [29]:
import json
import boto3
import botocore
from IPython.display import display, Markdown
import time

In [30]:
session = boto3.session.Session()
region = session.region_name
bedrock = boto3.client(service_name='bedrock-runtime', region_name=region)

In [34]:
# session # --> Session(region_name='ap-south-1')
# region  # --> 'ap-south-1'

'ap-south-1'

In [35]:
# Define model IDs that will be used in this module
MODELS = {
    "Titan Text G1 - Lite": "amazon.titan-text-lite-v1",
    "Titan Text G1 - Express": "amazon.titan-text-express-v1",

    # -- not having access for these models
    
    # "Claude 3.7 Sonnet": "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    # "Claude 3.5 Sonnet": "us.anthropic.claude-3-5-sonnet-20240620-v1:0",
    # "Claude 3.5 Haiku": "us.anthropic.claude-3-5-haiku-20241022-v1:0",
    # "Amazon Nova Pro": "us.amazon.nova-pro-v1:0",
    # "Amazon Nova Micro": "us.amazon.nova-micro-v1:0",
    # "DeepSeek-R1": "us.deepseek.r1-v1:0",
    # "Meta Llama 3.1 70B Instruct": "us.meta.llama3-1-70b-instruct-v1:0"
}

In [36]:
def display_response(response, model_name=None):
    if model_name:
        display(Markdown(f"### Response from {model_name}"))
    display(Markdown(response))
    print("\n" + "-"*80 + "\n")

In [37]:
text_to_summarize = """
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we're also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock's serverless experience, \
customers can easily find the right model for what they're trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
"""

In [39]:
# Create prompt for summarization
prompt1 = f"""Please provide a summary of the following text. Do not add any information that is not mentioned in the text below.
<text>
{text_to_summarize}
</text>
"""

In [40]:
prompt2 = "Explain me about AWS Bedrock"

In [41]:
selected_prompt = prompt1

In [42]:
titan_lite_body = json.dumps({
    "inputText": selected_prompt,
    "textGenerationConfig": {
        "maxTokenCount": 1000,
        "temperature": 1,
        "topP": 0.9
    }
})

In [43]:
titan_express_body = json.dumps({
    "inputText": selected_prompt,
    "textGenerationConfig": {
        "maxTokenCount": 1000,
        "temperature": 1,
        "topP": 0.9
    }
})

In [44]:
# Usage of 'invoke_model' api


titan_lite_response = bedrock.invoke_model(
        modelId=MODELS["Titan Text G1 - Lite"],
        body=titan_lite_body,
        accept="application/json",
        contentType="application/json"
    )
titan_lite_response_body = json.loads(titan_lite_response.get('body').read())

In [45]:
titan_express_response = bedrock.invoke_model(
        modelId=MODELS["Titan Text G1 - Express"],
        body=titan_express_body,
        accept="application/json",
        contentType="application/json"
    )
titan_express_response_body = json.loads(titan_express_response.get('body').read())

In [46]:
# print(response_body["outputText"])
print(titan_lite_response_body.get("results")[0]["outputText"])


Amazon Bedrock is a new service from AWS that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API, enabling customers to build and scale generative AI-based applications. It will offer the ability to access a range of powerful FMs for text and images through a scalable, reliable, and secure AWS managed service, with serverless experience that makes it easy to find the right model, get started quickly, customize FMs with their data, and integrate and deploy them into applications.


In [47]:
# print(response_body["outputText"])
print(titan_express_response_body.get("results")[0]["outputText"])


Amazon Bedrock is a new service that makes generative AI-based applications using FMs accessible via an API. It offers powerful FMs for text and images, including Amazons Titan FMs, through a scalable, reliable, and secure AWS managed service. Customers can easily find the right model, get started quickly, privately customize FMs with their own data, and easily integrate and deploy them into their applications. There is no need to manage any infrastructure, including integrations with Amazon SageMaker ML features.


In [49]:
#  ----------------------------------------------------- Use of 'converse' API's ----------------------------------------------

# Create a converse request with our summarization task
converse_request = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": f"Please provide a concise summary of the following text in 2-3 sentences. Text to summarize: {text_to_summarize}"
                }
            ]
        }
    ],
    "inferenceConfig": {
        "temperature": 0.4,
        "topP": 0.9,
        "maxTokens": 500
    }
}

In [73]:
# Call Titan Text G1 - Express Sonnet with Converse API
try:
    response = bedrock.converse(
        modelId=MODELS["Titan Text G1 - Express"],
        messages=converse_request["messages"],
        inferenceConfig=converse_request["inferenceConfig"]
    )
    
    # Extract the model's response
    titan_express_converse_response = response["output"]["message"]["content"][0]["text"]
    display_response(titan_express_converse_response, "Titan Text G1 - Express")
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Code']}: {error.response['Error']['Message']}\x1b[0m")
        print("Please ensure you have the necessary permissions for Amazon Bedrock.")
    else:
        raise error

### Response from Titan Text G1 - Express

Amazon Bedrock is a new service that makes AI-based applications using FMs accessible via an API, democratizing access for all builders. It offers a scalable, reliable, and secure AWS managed service with serverless experience, allowing customers to easily find the right model, get started quickly, privately customize FMs with their own data, and easily integrate and deploy them into their applications.


--------------------------------------------------------------------------------



In [54]:
# call different models with the same converse request
results = {}    
for model_name, model_id in MODELS.items(): # looping over all models defined above
        try:
            start_time = time.time()
            response = bedrock.converse(
                modelId=model_id,
                messages=converse_request["messages"],
                inferenceConfig=converse_request["inferenceConfig"] if "inferenceConfig" in converse_request else None
            )
            end_time = time.time()
            
            # Extract the model's response using the correct structure
            model_response = response["output"]["message"]["content"][0]["text"]
            response_time = round(end_time - start_time, 2)
            
            results[model_name] = {
                "response": model_response,
                "time": response_time
            }
            
            print(f"✅ Successfully called {model_name} (took {response_time} seconds)")
            
        except Exception as e:
            print(f"❌ Error calling {model_name}: {str(e)}")
            results[model_name] = {
                "response": f"Error: {str(e)}",
                "time": None
            }

✅ Successfully called Titan Text G1 - Lite (took 2.32 seconds)
✅ Successfully called Titan Text G1 - Express (took 2.92 seconds)


In [55]:
# Display results in a formatted way
for model_name, result in results.items():
    if "Error" not in result["response"]:
        display(Markdown(f"### {model_name} (took {result['time']} seconds)"))
        display(Markdown(result["response"]))
        print("-" * 80)

### Titan Text G1 - Lite (took 2.32 seconds)

Amazon Bedrock, a new service from AWS, makes AI21 Labs, Anthropic, Stability AI, and Amazon's FMs available through an API, democratizing access for all builders. It offers a scalable, reliable, and secure AWS managed service for customers to access a range of powerful FMs for text and images, including Amazons Titan FMs, through a serverless experience.

--------------------------------------------------------------------------------


### Titan Text G1 - Express (took 2.92 seconds)

Amazon Bedrock is a new service that makes AI-based applications accessible via an API, democratizing access for all builders. It offers a scalable, reliable, and secure AWS managed service with serverless experience, allowing customers to easily find the right model, get started quickly, privately customize FMs with their own data, and easily integrate and deploy them into their applications.

--------------------------------------------------------------------------------


In [ ]:
# # Regular model invocation (standard region)
# standard_response = bedrock.converse(
#     modelId="anthropic.claude-3-5-sonnet-20240620-v1:0",  # Standard model ID
#     messages=converse_request["messages"]
# )

# # Cross-region inference (note the "us." prefix)
# cris_response = bedrock.converse(
#     modelId="us.anthropic.claude-3-5-sonnet-20240620-v1:0",  # Cross-region model ID with regional prefix
#     messages=converse_request["messages"]
# )

# # Print responses
# print("Standard response:", standard_response["output"]["message"]["content"][0]["text"])
# print("Cross-region response:", cris_response["output"]["message"]["content"][0]["text"])

In [68]:
# Example of a multi-turn conversation with Converse API
multi_turn_messages = [
    {
        "role": "user",
        "content": [{"text": f"Please summarize this text: {text_to_summarize}"}]
    },
    {
        "role": "assistant",
        "content": [{"text": results["Titan Text G1 - Express"]["response"]}]
    },
    {
        "role": "user",
        "content": [{"text": "Can you make this summary even shorter, just 1 sentence?"}]
    }
]

try:
    response = bedrock.converse(
        modelId=MODELS["Titan Text G1 - Express"],
        messages=multi_turn_messages,
        inferenceConfig={"temperature": 0.2, "maxTokens": 500}
    )
    
    # Extract the model's response using the correct structure
    follow_up_response = response["output"]["message"]["content"][0]["text"]
    display_response(follow_up_response, "Titan Text G1 - Express (Multi-turn conversation)")
    
except Exception as e:
    print(f"Error: {str(e)}")

### Response from Titan Text G1 - Express (Multi-turn conversation)



Amazon Bedrock is a new service that makes AI-based applications accessible via an API, democratizing access for all builders.


--------------------------------------------------------------------------------



In [65]:
# Example of streaming with Converse API
def stream_converse(model_id, messages, inference_config=None):
    if inference_config is None:
        inference_config = {}
    
    print("Streaming response (chunks will appear as they are received):\n")
    print("-" * 80)
    
    full_response = ""
    
    try:
        response = bedrock.converse_stream(
            modelId=model_id,
            messages=messages,
            inferenceConfig=inference_config
        )
        response_stream = response.get('stream')
        if response_stream:
            for event in response_stream:

                if 'messageStart' in event:
                    print(f"\nRole: {event['messageStart']['role']}")

                if 'contentBlockDelta' in event:
                    print(event['contentBlockDelta']['delta']['text'], end="")

                if 'messageStop' in event:
                    print(f"\nStop reason: {event['messageStop']['stopReason']}")

                if 'metadata' in event:
                    metadata = event['metadata']
                    if 'usage' in metadata:
                        print("\nToken usage")
                        print(f"Input tokens: {metadata['usage']['inputTokens']}")
                        print(
                            f":Output tokens: {metadata['usage']['outputTokens']}")
                        print(f":Total tokens: {metadata['usage']['totalTokens']}")
                    if 'metrics' in event['metadata']:
                        print(
                            f"Latency: {metadata['metrics']['latencyMs']} milliseconds")

                
            print("\n" + "-" * 80)
        return full_response
    
    except Exception as e:
        print(f"Error in streaming: {str(e)}")
        return None

In [66]:
# Let's try streaming a longer summary
streaming_request = [
    {
        "role": "user",
        "content": [
            {
                "text": f"""Please provide a detailed summary of the following text, explaining its key points and implications:
                
                {text_to_summarize}
                
                Make your summary comprehensive but clear.
                """
            }
        ]
    }
]

In [69]:
# Only run this when you're ready to see streaming output
streamed_response = stream_converse(
    MODELS["Titan Text G1 - Express"], 
    streaming_request, 
    inference_config={"temperature": 0.4, "maxTokens": 1000}
)

Streaming response (chunks will appear as they are received):

--------------------------------------------------------------------------------

Role: assistant

Amazon Bedrock is a new service that makes AI-based applications using FMs accessible via an API, democratizing access for all builders. It offers a range of powerful FMs for text and images, including Amazons Titan FMs, through a scalable, reliable, and secure AWS managed service. With serverless experience, customers can easily find the right model for what they're trying to get done, get started quickly, privately customize FMs with their own data, and easily integrate and deploy them into their applications using AWS tools and capabilities.
Stop reason: end_turn

Token usage
Input tokens: 250
:Output tokens: 111
:Total tokens: 361
Latency: 4407 milliseconds

--------------------------------------------------------------------------------


In [ ]:
#     """Mock function that would normally call a weather API"""
#     print(f"Looking up weather for {location}...")
    
#     # In a real application, this would call a weather API
#     weather_data = {
#         "New York": {"condition": "Partly Cloudy", "temperature": 72, "humidity": 65},
#         "San Francisco": {"condition": "Foggy", "temperature": 58, "humidity": 80},
#         "Miami": {"condition": "Sunny", "temperature": 85, "humidity": 75},
#         "Seattle": {"condition": "Rainy", "temperature": 52, "humidity": 90}
#     }
    
#     return weather_data.get(location, {"condition": "Unknown", "temperature": 0, "humidity": 0})

In [74]:
weather_tool = {
    "tools": [
        {
            "toolSpec": {
                "name": "get_weather",
                "description": "Get current weather for a specific location",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city name to get weather for"
                            }
                        },
                        "required": ["location"]
                    }
                }
            }
        }
    ],
    "toolChoice": {
        "auto": {}  # Let the model decide when to use the tool
    }
}

In [71]:
function_request = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": "What's the weather like in San Francisco right now? And what should I wear?"
                }
            ]
        }
    ],
    "inferenceConfig": {
        "temperature": 0.0,  # Use 0 temperature for deterministic function calling
        "maxTokens": 500
    }
}

In [75]:
# need to have access for this 'Claude 3.7 Sonnet' model


response = bedrock.converse(
    modelId=MODELS["Claude 3.7 Sonnet"],
    messages=function_request["messages"],
    inferenceConfig=function_request["inferenceConfig"],
    toolConfig=weather_tool
)
print(json.dumps(response, indent=2))

KeyError: 'Claude 3.7 Sonnet'

In [76]:
def handle_function_calling(model_id, request, tool_config):
    try:
        # Step 1: Send initial request
        response = bedrock.converse(
            modelId=model_id,
            messages=request["messages"],
            inferenceConfig=request["inferenceConfig"],
            toolConfig=tool_config
        )
        
        # Check if the model wants to use a tool (check the correct response structure)
        content_blocks = response["output"]["message"]["content"]
        has_tool_use = any("toolUse" in block for block in content_blocks)
        
        if has_tool_use:
            # Find the toolUse block
            tool_use_block = next(block for block in content_blocks if "toolUse" in block)
            tool_use = tool_use_block["toolUse"]
            tool_name = tool_use["name"]
            tool_input = tool_use["input"]
            tool_use_id = tool_use["toolUseId"]
            
            # Step 2: Execute the tool
            if tool_name == "get_weather":
                tool_result = get_weather(tool_input["location"])
            else:
                tool_result = {"error": f"Unknown tool: {tool_name}"}
            
            # Step 3: Send the tool result back to the model
            updated_messages = request["messages"] + [
                {
                    "role": "assistant",
                    "content": [
                        {
                            "toolUse": {
                                "toolUseId": tool_use_id,
                                "name": tool_name,
                                "input": tool_input
                            }
                        }
                    ]
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "toolResult": {
                                "toolUseId": tool_use_id,
                                "content": [
                                    {
                                        "json": tool_result
                                    }
                                ],
                                "status": "success"
                            }
                        }
                    ]
                }
            ]
            
            # Step 4: Get final response
            final_response = bedrock.converse(
                modelId=model_id,
                messages=updated_messages,
                inferenceConfig=request["inferenceConfig"],
                toolConfig=tool_config  
            )
            
            # Extract text from the correct response structure
            final_text = ""
            for block in final_response["output"]["message"]["content"]:
                if "text" in block:
                    final_text = block["text"]
                    break
            
            return {
                "tool_call": {"name": tool_name, "input": tool_input},
                "tool_result": tool_result,
                "final_response": final_text
            }
        else:
            # Model didn't use a tool, just return the text response
            text_response = ""
            for block in content_blocks:
                if "text" in block:
                    text_response = block["text"]
                    break
                    
            return {
                "final_response": text_response
            }
    
    except Exception as e:
        print(f"Error in function calling: {str(e)}")
        return {"error": str(e)}